In [1]:
import pandas as pd
import numpy as np

from permacache import permacache

In [2]:
%matplotlib inline

In [45]:
orig = data = pd.read_csv("https://elections.cdn.sos.ca.gov/sov/2021-recall/sov/csv-candidates.csv")
data = data[data["Contest Name"] == "Governor"].copy()
data["vote_total"] = data["Vote Total"].apply(lambda x: int(x.replace(",", "")))
data = data[["County Id", "County Name", "Candidate Name", "vote_total"]]
data = data.pivot(index="County Id", columns="Candidate Name", values="vote_total")

In [90]:
def load(f):
    data = pd.read_excel(f)[1::3].set_index("Unnamed: 0")
    data = data[[x for x in data if "Unnamed" not in x]]
    data = data.applymap(int)
    totals = data.T.sum()
    padillas = data["Alex \nPadilla"]
    return data, padillas / totals

In [91]:
special, s = load("./80-uss-pt.xlsx")
general, g = load("./68-uss-ft.xlsx")

In [92]:
s - g

Unnamed: 0
Alameda            0.007895
Alpine            -0.002955
Amador             0.010774
Butte             -0.033375
Calaveras          0.024988
Colusa             0.020538
Contra Costa       0.005992
Del Norte         -0.009209
El Dorado          0.009716
Fresno             0.016627
Glenn             -0.014585
Humboldt          -0.007923
Imperial           0.035235
Inyo               0.016205
Kern               0.011424
Kings              0.014790
Lake               0.033566
Lassen            -0.002976
Los Angeles        0.028480
Madera             0.028673
Marin             -0.001953
Mariposa           0.021625
Mendocino         -0.022149
Merced             0.011929
Modoc              0.007105
Mono               0.003478
Monterey           0.019484
Napa               0.016714
Nevada            -0.000936
Orange            -0.008037
Placer            -0.006605
Plumas            -0.002474
Riverside          0.003872
Sacramento         0.012217
San Benito         0.008405
San Berna

In [111]:
result = (s - g) * 100
# def convert(x):
#     if x < -50:
#         return "> J+50"
#     if x < -25:
#         return "J+25 - J+50"
#     if x < -13:
#         return "J+13 - J+25"
#     if x < -6:
#         return "J+6 - J+13"
#     if x < -3:
#         return "J+3 - J+6"
#     if x < 0:
#         return "<J+3"
#     return convert(-x).replace("J", "D")
# result = result.apply(convert)

In [112]:
@permacache("county_importance_wikipedia/load_county_geojson_3")
def load_county_geojson():
    import tempfile, os, requests, geopandas
    tempdir = tempfile.TemporaryDirectory()
    rootpath = tempdir.name
    os.system(f"mkdir -p {rootpath}")
    zip = requests.get(
        "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_500k.zip"
    ).content
    with open(f"{rootpath}/hi.zip", "wb") as f:
        f.write(zip)
    os.system(f"cd {rootpath}; unzip hi.zip")

    return geopandas.read_file(f"{rootpath}/cb_2020_us_county_500k.shp")

In [113]:
# back_name = {name : i for i, name in named.items()}

geojson = load_county_geojson()
geojson = geojson[geojson["STATEFP"] == "06"].copy()
geojson["margin"] = geojson.NAME.apply(lambda x: result[x])
geojson.to_file("temp/out.shp")

In [114]:
geojson

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,margin
9,06,037,00277283,0500000US06037,06037,Los Angeles,Los Angeles County,CA,California,06,10513491099,1787501506,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",2.848019
10,06,023,01681908,0500000US06023,06023,Humboldt,Humboldt County,CA,California,06,9241565229,1253726036,"POLYGON ((-124.40860 40.44320, -124.39664 40.4...",-0.792327
11,06,097,01657246,0500000US06097,06097,Sonoma,Sonoma County,CA,California,06,4080862341,498084088,"POLYGON ((-123.53306 38.76842, -123.52851 38.7...",-0.253498
12,06,079,00277304,0500000US06079,06079,San Luis Obispo,San Luis Obispo County,CA,California,06,8549172832,815618272,"POLYGON ((-121.34636 35.79518, -121.24498 35.7...",-0.526570
19,06,089,01682610,0500000US06089,06089,Shasta,Shasta County,CA,California,06,9778551576,186157964,"POLYGON ((-123.06884 40.30826, -123.06734 40.3...",0.405291
36,06,039,00277284,0500000US06039,06039,Madera,Madera County,CA,California,06,5534591552,42271836,"POLYGON ((-120.54554 37.02758, -120.54398 37.0...",2.867346
332,06,081,00277305,0500000US06081,06081,San Mateo,San Mateo County,CA,California,06,1161922036,757123906,"POLYGON ((-122.52085 37.59418, -122.51533 37.5...",-1.459568
333,06,073,00277301,0500000US06073,06073,San Diego,San Diego County,CA,California,06,10904455257,817317026,"POLYGON ((-117.59600 33.38779, -117.59360 33.3...",0.885764
334,06,015,01682074,0500000US06015,06015,Del Norte,Del Norte County,CA,California,06,2606118035,578742633,"MULTIPOLYGON (((-124.21749 41.95081, -124.2170...",-0.920883
335,06,093,00277311,0500000US06093,06093,Siskiyou,Siskiyou County,CA,California,06,16261933231,179149816,"POLYGON ((-123.71845 41.59797, -123.71880 41.5...",-1.908678


In [115]:
result

Unnamed: 0
Alameda            0.789508
Alpine            -0.295470
Amador             1.077395
Butte             -3.337497
Calaveras          2.498817
Colusa             2.053750
Contra Costa       0.599208
Del Norte         -0.920883
El Dorado          0.971582
Fresno             1.662747
Glenn             -1.458498
Humboldt          -0.792327
Imperial           3.523528
Inyo               1.620487
Kern               1.142382
Kings              1.479039
Lake               3.356591
Lassen            -0.297628
Los Angeles        2.848019
Madera             2.867346
Marin             -0.195254
Mariposa           2.162517
Mendocino         -2.214921
Merced             1.192877
Modoc              0.710524
Mono               0.347780
Monterey           1.948417
Napa               1.671401
Nevada            -0.093617
Orange            -0.803709
Placer            -0.660462
Plumas            -0.247441
Riverside          0.387219
Sacramento         1.221709
San Benito         0.840478
San Berna

In [116]:
s

Unnamed: 0
Alameda            0.733101
Alpine             0.575290
Amador             0.339782
Butte              0.386331
Calaveras          0.337177
Colusa             0.275569
Contra Costa       0.625900
Del Norte          0.352012
El Dorado          0.386931
Fresno             0.421188
Glenn              0.223232
Humboldt           0.557995
Imperial           0.514510
Inyo               0.402234
Kern               0.319430
Kings              0.303869
Lake               0.449649
Lassen             0.175647
Los Angeles        0.644604
Madera             0.322376
Marin              0.761475
Mariposa           0.358542
Mendocino          0.578048
Merced             0.403287
Modoc              0.202077
Mono               0.501748
Monterey           0.596361
Napa               0.610065
Nevada             0.506302
Orange             0.434332
Placer             0.379069
Plumas             0.343806
Riverside          0.429520
Sacramento         0.541088
San Benito         0.494035
San Berna

In [121]:
general.iloc[-1]

Akinyemi \nAgbede                 70971
Dan \nO'Dowd                      74916
Alex \nPadilla                  3725544
Douglas \nHoward \nPierce        116771
Obaidul\n Huq \nPirjada           27889
Timothy J \nUrsich                58348
James P. \nBradley               235788
Jon \nElist                      289716
Myron L. \nHall                   66161
Sarah \nSun \nLiew                76994
Robert \nGeorge \nLucero, Jr      53398
Mark P. \nMeuser                1028374
Enrique \nPetris                  31883
Chuck \nSmith                    266766
Carlos \nGuillermo \nTapia        33870
Cordie \nWilliams                474321
James \n"Henk" \nConn             35983
Pamela \nElizondo                 31981
John \nThompson \nParker         105477
Daphne \nBradford                 26900
Eleanor \nGarcia                  34625
Don J.\nGrundmann                 10181
Deon D. \nJenkins                  6936
Lijun \n(Lily)\n Zhou                58
Irene \nRatliff                       7


In [122]:
g

Unnamed: 0
Alameda            0.725206
Alpine             0.578244
Amador             0.329008
Butte              0.419706
Calaveras          0.312189
Colusa             0.255032
Contra Costa       0.619908
Del Norte          0.361221
El Dorado          0.377215
Fresno             0.404560
Glenn              0.237817
Humboldt           0.565918
Imperial           0.479275
Inyo               0.386029
Kern               0.308006
Kings              0.289078
Lake               0.416083
Lassen             0.178623
Los Angeles        0.616124
Madera             0.293702
Marin              0.763428
Mariposa           0.336916
Mendocino          0.600197
Merced             0.391358
Modoc              0.194972
Mono               0.498270
Monterey           0.576877
Napa               0.593351
Nevada             0.507238
Orange             0.442369
Placer             0.385673
Plumas             0.346280
Riverside          0.425647
Sacramento         0.528871
San Benito         0.485630
San Berna

In [129]:
s - g

Unnamed: 0
Alameda            False
Alpine              True
Amador             False
Butte              False
Calaveras          False
Colusa             False
Contra Costa       False
Del Norte          False
El Dorado          False
Fresno             False
Glenn              False
Humboldt           False
Imperial           False
Inyo               False
Kern               False
Kings              False
Lake               False
Lassen              True
Los Angeles        False
Madera             False
Marin               True
Mariposa           False
Mendocino          False
Merced             False
Modoc              False
Mono               False
Monterey           False
Napa               False
Nevada              True
Orange             False
Placer             False
Plumas              True
Riverside          False
Sacramento         False
San Benito         False
San Bernardino      True
San Diego          False
San Francisco      False
San Joaquin        False
San Luis Obisp